### Edgar Moises Hernandez-Gonzalez
#### 19/11/19-21/11/19
#### Clasificacion de RWE-DWT de MI-EEG con CNN-1D
##### Validacion cruzada de 5
##### Optimizar hiperparametros con busqueda de cuadricula

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout
from tensorflow.keras.wrappers import scikit_learn
from sklearn.model_selection import GridSearchCV

In [0]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/MI-EEG-RWE-A01T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/MI-EEG-RWE-A01E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/EtiquetasA01T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/EtiquetasA01E.csv", header=None)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 66)
(288, 66)
(288, 1)
(288, 1)


In [6]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

print(x.shape)
print(y.shape)

(576, 66)
(576, 1)


In [7]:
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [8]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [0]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
scaler = StandardScaler().fit(x)
x = scaler.transform(x)

In [10]:
#codificar etiquetas con valores de 0 a nClases-1
le = LabelEncoder().fit(y)
y = le.transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
#convertir de 2D a 3D
x = x.reshape((n_samples, 3, 22))

print(x.shape)

(576, 3, 22)


In [0]:
def model(n_filtros, n_neuronas, p_dropout):
    model = Sequential()
    model.add(Conv1D(n_filtros, 3, activation='relu', input_shape=(3,22)))
    model.add(Flatten())
    model.add(Dense(n_neuronas, activation='relu'))
    model.add(Dropout(p_dropout))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [0]:
clf = scikit_learn.KerasClassifier(build_fn = model, epochs=20, batch_size=18, verbose=0)

In [0]:
n_filtros = [16, 32, 64]
n_neuronas = [16, 32, 64, 128]
p_dropout = [0.2, 0.5]
hiperparametros = dict(n_filtros=n_filtros, n_neuronas=n_neuronas, p_dropout=p_dropout)

In [0]:
clf = GridSearchCV(clf, hiperparametros, cv=5)

In [16]:
clf.fit(x, y)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


KeyboardInterrupt: ignored

In [0]:
print(clf.best_params_)
print(clf.best_score_)

{'n_filtros': 32, 'n_filtros_2': 64, 'n_neuronas': 16, 'p_dropout': 0.2}
0.4999999962747097


In [0]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.453 (+/-0.067) for {'n_filtros': 16, 'n_filtros_2': 32, 'n_neuronas': 16, 'p_dropout': 0.2}
0.429 (+/-0.072) for {'n_filtros': 16, 'n_filtros_2': 32, 'n_neuronas': 16, 'p_dropout': 0.5}
0.458 (+/-0.077) for {'n_filtros': 16, 'n_filtros_2': 32, 'n_neuronas': 32, 'p_dropout': 0.2}
0.443 (+/-0.050) for {'n_filtros': 16, 'n_filtros_2': 32, 'n_neuronas': 32, 'p_dropout': 0.5}
0.441 (+/-0.103) for {'n_filtros': 16, 'n_filtros_2': 32, 'n_neuronas': 64, 'p_dropout': 0.2}
0.441 (+/-0.066) for {'n_filtros': 16, 'n_filtros_2': 32, 'n_neuronas': 64, 'p_dropout': 0.5}
0.460 (+/-0.076) for {'n_filtros': 16, 'n_filtros_2': 32, 'n_neuronas': 128, 'p_dropout': 0.2}
0.453 (+/-0.090) for {'n_filtros': 16, 'n_filtros_2': 32, 'n_neuronas': 128, 'p_dropout': 0.5}
0.453 (+/-0.099) for {'n_filtros': 16, 'n_filtros_2': 64, 'n_neuronas': 16, 'p_dropout': 0.2}
0.413 (+/-0.091) for {'n_filtros': 16, 'n_filtros_2': 64, 'n_neuronas': 16, 'p_dropout': 0.5}
0.446 (+/-0.028) for {'n_filtros': 16, 'n_filtros_2': 64, 